In [1]:
import pandas as pd

In [2]:
muni16 = pd.read_csv('E:/Transit-Casa-Alex/Output/Processed MUNI Rail/Mapping_Data_16.csv',thousands = ',')

In [3]:
muni09 = pd.read_csv('E:/Transit-Casa-Alex/Output/Processed MUNI Rail/2009_MUNI_Rail.csv',thousands = ',')

In [4]:
muni09.head()

,Boardings,Alightings,STOP_ID
0,132,58,3537
1,60,170,3538
2,726,810,7316
3,198,239,3985
4,286,340,3987


In [45]:
muni16.head()

,STOP NAME,STOP ID,Total Boardings,Total Alightings,Stop Latitude,Stop Longitude
0,9th Ave & Irving St,3212,4.0,2.1,37.763912,-122.46624
1,9th Ave & Irving St,3212,248.0,70.6,37.763912,-122.46624
2,9th Ave & Irving St,3212,247.7,84.2,37.763912,-122.46624
3,9th Ave & Irving St,3212,264.9,62.6,37.763912,-122.46624
4,9th Ave & Irving St,3212,208.3,49.1,37.763912,-122.46624


In [46]:
muni16.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2127 entries, 0 to 2126
Data columns (total 6 columns):
STOP NAME           2127 non-null object
STOP ID             2127 non-null int64
Total Boardings     1858 non-null float64
Total Alightings    1862 non-null float64
Stop Latitude       2127 non-null float64
Stop Longitude      2127 non-null float64
dtypes: float64(4), int64(1), object(1)
memory usage: 99.8+ KB


In [102]:
muni09_group = muni09.groupby(by = 'STOP_ID',as_index = False).sum()

In [103]:
muni16_group = muni16.groupby(by = 'STOP ID',as_index = False).agg({'STOP NAME':'first','Stop Latitude':'mean','Stop Longitude':'mean','Total Alightings':'sum','Total Boardings':'sum'})

In [105]:
muni16_group.columns

Index(['STOP ID', 'Stop Latitude', 'Stop Longitude', 'STOP NAME',
       'Total Alightings', 'Total Boardings'],
      dtype='object')

In [106]:
rename = ['STOP_ID', 'LAT', 'LON', 'STOP_NAME',
       'Alightings', 'Boardings']

In [107]:
muni16_group.columns = rename

In [108]:
muni16_group.head()

,STOP_ID,LAT,LON,STOP_NAME,Alightings,Boardings
0,3212,37.763912,-122.466240,9th Ave & Irving St,292.4,1046.3
1,3266,37.743122,-122.471300,15th Ave & Taraval St,313.8,119.6
2,3267,37.743070,-122.471405,15th Ave & Taraval St,92.3,240.7
3,3268,37.741524,-122.471190,15th Ave & Ulloa St,233.4,16.5
4,3269,37.741486,-122.471300,15th Ave & Ulloa St,23.3,90.4


In [109]:
rail = pd.merge(muni16_group,muni09_group,on = 'STOP_ID',how = 'inner',suffixes = ('_16','_09'))

In [110]:
rail[rail['STOP_ID'] == 3537]

,STOP_ID,LAT,LON,STOP_NAME,Alightings_16,Boardings_16,Boardings_09,Alightings_09
10,3537,37.742306,-122.42403,30th St & Dolores St,101.4,249.0,132,58


In [111]:
muni16_group[muni16_group['STOP_ID'] == 3537]

,STOP_ID,LAT,LON,STOP_NAME,Alightings,Boardings
10,3537,37.742306,-122.42403,30th St & Dolores St,101.4,249.0


In [112]:
rail['DIFF'] = round(rail['Boardings_16'].astype(float) - rail['Boardings_09'].astype(float))

In [113]:
rail['P_DIFF'] = round((rail['DIFF']/rail['Boardings_09'].astype(float))*100)

In [114]:
rail['P_DIFF_STR'] = rail['P_DIFF'].apply(lambda value: str(value) + '%')

In [115]:
rail2 = rail[['STOP_ID','Alightings_09','Boardings_09','STOP_NAME','Boardings_16','Alightings_16','DIFF','P_DIFF']].fillna(0)
rail['STOP_ID'] = rail2.STOP_ID
rail.Boardings_09 = rail2.Boardings_09
rail.Alightings_09 = rail2.Alightings_09
rail.STOP_NAME = rail2.STOP_NAME
rail.Boardings_16 = rail2.Boardings_16
rail.Alightings_16 = rail2.Alightings_16
rail.DIFF = rail2.DIFF
rail.P_DIFF = rail2.P_DIFF
rail.LAT = rail.LAT.astype(float)
rail.head()

,STOP_ID,LAT,LON,STOP_NAME,Alightings_16,Boardings_16,Boardings_09,Alightings_09,DIFF,P_DIFF,P_DIFF_STR
0,3212,37.763912,-122.466240,9th Ave & Irving St,292.4,1046.3,1223,270,-177.0,-14.0,-14.0%
1,3266,37.743122,-122.471300,15th Ave & Taraval St,313.8,119.6,77,255,43.0,56.0,56.0%
2,3267,37.743070,-122.471405,15th Ave & Taraval St,92.3,240.7,280,77,-39.0,-14.0,-14.0%
3,3268,37.741524,-122.471190,15th Ave & Ulloa St,233.4,16.5,22,111,-6.0,-27.0,-27.0%
4,3269,37.741486,-122.471300,15th Ave & Ulloa St,23.3,90.4,47,13,43.0,91.0,91.0%


In [120]:
rail.to_csv('E:/Transit-Casa-Alex/Output/Processed MUNI Rail/MUNI_Rail_Map_Data.csv')

In [117]:
rail.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 236 entries, 0 to 235
Data columns (total 11 columns):
STOP_ID          236 non-null int64
LAT              236 non-null float64
LON              236 non-null float64
STOP_NAME        236 non-null object
Alightings_16    236 non-null float64
Boardings_16     236 non-null float64
Boardings_09     236 non-null int64
Alightings_09    236 non-null int64
DIFF             236 non-null float64
P_DIFF           236 non-null float64
P_DIFF_STR       236 non-null object
dtypes: float64(6), int64(3), object(2)
memory usage: 22.1+ KB


In [118]:
import numpy as np
rail[rail['P_DIFF'] == 0]

,STOP_ID,LAT,LON,STOP_NAME,Alightings_16,Boardings_16,Boardings_09,Alightings_09,DIFF,P_DIFF,P_DIFF_STR
81,5221,37.760365,-122.50901,Judah/La Playa/Ocean Beach,518.7,0.0,0,701,0.0,0.0,nan%
223,7393,37.732280,-122.39153,Third Street/Revere/Shafter,428.5,185.3,185,421,0.0,0.0,0.0%
226,7396,37.708970,-122.40511,Bay Shore Blvd & Sunnydale Ave,182.5,0.0,0,561,0.0,0.0,nan%


In [119]:
rail.STOP_ID.value_counts()

5119    1
6280    1
5786    1
5726    1
5784    1
6275    1
5780    1
5779    1
6288    1
6287    1
6285    1
3212    1
6217    1
7343    1
3719    1
5788    1
3718    1
3717    1
3716    1
3715    1
6274    1
5204    1
6269    1
6268    1
6266    1
6265    1
5240    1
5239    1
5787    1
3998    1
       ..
3376    1
3385    1
6637    1
7363    1
3985    1
3984    1
5118    1
7252    1
5793    1
7365    1
6503    1
5203    1
4448    1
4447    1
6633    1
7342    1
6998    1
6997    1
5201    1
6995    1
6994    1
6993    1
6992    1
6991    1
3403    1
3913    1
3912    1
5198    1
3909    1
5120    1
Name: STOP_ID, dtype: int64